In [ ]:
import tensorflow as tf
import numpy as np

from scipy.io import loadmat

#### 1) Read data
Read the matlab generated `mat` file

In [ ]:
path = 'data_alpha0.65_beta0.75_r1_conf_th0.01_edge_size200.mat'
image_shape = [1080, 1920]
iou_threshold = 0.5
data_dict = loadmat(path)['data']

#### 2) Populate
Populate a list with all the entries

In [ ]:
gt_boxes = []
predicted_boxes = []
proj_matrices = []

max_gt = 0
max_predicted = 0

for i in range(data_dict.shape[0]):
    gt_boxes.append(data_dict[i][0][1])
    predicted_boxes.append(data_dict[i][0][2][:, :4])
    proj_matrices.append(data_dict[i][0][3])
    
    if max_gt < gt_boxes[-1].shape[0]:
        max_gt = gt_boxes[-1].shape[0]
        
    if max_predicted < predicted_boxes[-1].shape[0]:
        max_predicted = predicted_boxes[-1].shape[0]

#### 3) Pad
Fill out a matrix with the predicted boxes and ground truth appended with -1s according to their maximum sizes

In [ ]:
predicted_boxes_padded = []
for i in range(len(predicted_boxes)):
    temp_boxes = predicted_boxes[i]
    padded = - np.ones((max_predicted, 4))
    if predicted_boxes[i].shape[0]:
        padded[:temp_boxes.shape[0], :] = temp_boxes
    predicted_boxes_padded.append(padded)
predicted_boxes = predicted_boxes_padded

gt_boxes_padded = []
for i in range(len(gt_boxes)):
    temp_boxes = gt_boxes[i]
    padded = - np.ones((max_gt, 4))
    if gt_boxes[i].shape[0] > 0:
        padded[:temp_boxes.shape[0], :] = temp_boxes
    gt_boxes_padded.append(padded)
gt_boxes = gt_boxes_padded

predicted_boxes = np.stack(predicted_boxes)
gt_boxes = np.stack(gt_boxes)
proj_matrices = np.stack(proj_matrices)

#### 4) Tensors!
Set the tensors in prepearation of the evaluation

In [ ]:
predicted_tensor = tf.constant(predicted_boxes, dtype=tf.float32)
gt_tensor = dict(boxes=tf.constant(gt_boxes, dtype=tf.float32), proj_matrix=tf.constant(proj_matrices, dtype=tf.float32))

#### 5a) Apply `DetectionEvaluator` from beatrix

In [ ]:
from hudl_beatrix.evaluators import ModModelEvaluator

In [ ]:
evaluator = ModModelEvaluator(iou_threshold=iou_threshold)
log_results_tensor = evaluator(gt_tensor, predicted_tensor)

In [ ]:
with tf.Session() as s:
    stats = s.run(log_results_tensor)

print(stats)

#### 5b) Apply `WatsonFilteringEvaluator` from beatrix

In [ ]:
from hudl_beatrix.evaluators.model_wrappers.watson_filtering import WatsonFilteringModelEvaluator

In [ ]:
evaluator = WatsonFilteringModelEvaluator(iou_threshold=iou_threshold)
stats_tensor = evaluator(gt_tensor, predicted_tensor, 
                         input_height=image_shape[0],
                         input_width=image_shape[1])

In [ ]:
with tf.Session() as s:
    stats = s.run(stats_tensor)

print(stats)